In [1]:
# !pip install --upgrade tensorflow --quiet
# !pip install keras_tuner --quiet
# !pip install tensorflow-io --quiet
# # Google colab modules
# from google.colab import drive
import sys, importlib

# # Mount drive
# drive.mount('/content/gdrive', force_remount=True)
ROOT_PATH = './'
# sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()
# core._init_models()
# core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

In [2]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


### model_323_kt - world stock data (small)

In [2]:
kt_iter = '26'
ticker_group = ['wsd']
epochs = 20
max_epochs = 20
batch_size = 64

In [3]:
dataset_size = 'small'
shuffle_buffer_size = 1024
generator = False

train_inputs, train_labels = core.load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator)

validation_inputs, validation_labels = core.load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator)

Total constituents: 22
Total constituents: 8


In [4]:
tuner = kt.Hyperband(hypermodel=core.model_323_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), epochs=epochs, verbose=1)

Trial 30 Complete [00h 30m 10s]
val_loss: 0.6954605579376221

Best val_loss So Far: 0.6693868637084961
Total elapsed time: 05h 08m 03s


In [5]:
kt_iter = '28'
ticker_group = ['wsd']
epochs = 20
max_epochs = 20
batch_size = 64

dataset_size = 'small'
shuffle_buffer_size = 1024
generator = False

train_inputs, train_labels = core.load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator)

validation_inputs, validation_labels = core.load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator)

tuner = kt.Hyperband(hypermodel=core.model_325_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), epochs=epochs, verbose=1)

Trial 30 Complete [00h 26m 20s]
val_loss: 0.6987267732620239

Best val_loss So Far: 0.6756389141082764
Total elapsed time: 05h 46m 26s


### 325/31 v8 full_new_wsd

In [ ]:
model_no = '325'
kt_iter = '31'
ticker_group = ['wsd']
epochs = 20
max_epochs = 20
batch_size = 64

dataset_size = 'full_new_wsd'
shuffle_buffer_size = 2048
generator = False

train_inputs, train_labels = core.load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

validation_inputs, validation_labels = core.load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

tuner = kt.Hyperband(hypermodel=core.model_325_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), epochs=epochs, verbose=1)

Trial 10 Complete [00h 12m 10s]
val_loss: 0.6963037252426147

Best val_loss So Far: 0.6912412643432617
Total elapsed time: 02h 01m 34s

Search: Running Trial #11

Hyperparameter    |Value             |Best Value So Far 
lr                |4.1844e-05        |2.4351e-05        
c_filters         |48                |64                
r_units           |112               |112               
tuner/epochs      |3                 |3                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/3
5345/5345 [==============================] - 244s 44ms/step - loss: 0.6783 - accuracy: 0.5427 - val_loss: 0.6927 - val_accuracy: 0.5307
Epoch 2/3
5345/5345 [==============================] - 231s 43ms/step - loss: 0.6614 - accuracy: 0.5755 - val_loss: 0.6919 - val_accuracy: 0.5307
Epoch 3/3
1827/5345 [=========>....................] - ETA: 1:52 - loss: 0.6870 - accura

### Run with revised generator
- Use 326 as early benchmark model, with world portion variation to find the optimal point

In [ ]:
# RERUN WITH REVISED traintest slice code
model_no = '326'
constituent_limits = 0.05
id_constituent = 1
kt_iter = f'_model-{model_no}_wc-{constituent_limits}_ic-{id_constituent}'
ticker_group = ['wsd']
epochs = 40
max_epochs = 40
batch_size = 2048
shuffle_buffer_size = 16
generator = True

# Train: `slice_from_beginning`=True
train_gen = core.load_dataset_wsd(slice_from_beginning=True, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)
# Validation: `slice_from_beginning`=False. constituent_limits in validation is always 0 (focus on idx performance progression only)
validation_gen = core.load_dataset_wsd(slice_from_beginning=False, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', constituent_limits=0, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

tuner = kt.Hyperband(hypermodel=core.model_326_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

Total constituents: 3386
Total constituents: 422

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
lr                |0.0082852         |?                 
r_units           |112               |?                 
d_units_1         |64                |?                 
d_units_2         |64                |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |3                 |?                 
tuner/round       |0                 |?                 

